In [ ]:
from json import dumps
from os.path import abspath
from re import sub

# Splits each line (a path representation) in the text file into a list of path elements.
# We can use the length of the list to determine the nesting depth of each OU
org_ou_file = './tests/organization_ous.txt'
org_ou_paths = []
with open(abspath(org_ou_file)) as f:
    org_ou_paths = f.readlines()
org_ou_path_list = [sub('^\/', '', ou_path).rstrip('\n').split('/') for ou_path in org_ou_paths]
print(dumps(org_ou_path_list, indent=4))

In [ ]:
# {
#   node1 = [child1, child2, child3],
#   node2 = [child1, child2, child3],
#   note3 = [child1, child2, child3]
# }

# - create a UUID for each node. since we can't guarantee path uniqueness due to mutability,
#   the UUID can be used to track notes prior to getting an ou_id from the Organizations API.
# - paths with single elements are root OUs
# - paths with more than one element are nested OUs
# - create the OUs starting with the roots inwards
# - 


# Unique, non-changing hash for each OU
# - Input OU ARN (guaranteed not to change)
# - Generate a sha1 hash
# - Trim the hash to 8 chars

In [ ]:
def get_root_ous(ou_path_list: list[str]):
    if len(ou_path_list) == 0:
        raise Exception('Received an empty list')
    root_ous = []
    for ou_path in ou_path_list:
        if len(ou_path) == 0:
            print('Warning: OU path element is empty. Skipping.')
            continue
        root_ous.append(ou_path[0])
    return list(set(root_ous))

In [ ]:
root_ous = get_root_ous(org_ou_path_list)
print(dumps(root_ous))

In [ ]:
from hashlib import sha1

def gen_sha1_id(id_str: str, length: int = 6):
    return sha1(id_str.encode()).hexdigest()[0:length]

In [ ]:
from aws_data_tools import models
org = models.AWSOrganization.init()
org.get_ous()
print(f'Root ID: {org.root_id}')
print(f'First OU: {org.ous[0]}')

In [ ]:
# root_ous = {gen_sha1_id(ou['id']): ou for ou in org.ous}
root_ous = {ou['id']: ou for ou in org.ous}
root_ous


In [ ]:
ous_raw = root_ous_raw

unvisited_nodes = org.ous

# Create initial digraph with the root ous
org_digraph = { org.root_id: org.ous }

In [ ]:
for node in unvisited_nodes:
    children = org.list_children_for_parent(parent_id=node['id'], child_type='organizational_unit')
    org_digraph[node['id']] = children

In [ ]:
ous = {
    'lvl_1': org.get_ous()
}

for 


_ = [
    {**ou, **{"children": org.list_children_for_parent(parent_id=ou["id"], child_type="organizational_unit")}}
    for ou in org.ous
]



In [ ]:
[{x['name']: len(x['children'])} for x in ous_with_children_lvl_1]

# DFS Queue Method

In [23]:
from aws_data_tools.models import (
    AWSOrganization,
    AWSOrganizationRoot,
    AWSOrganizationRoots,
    AWSOrganizationOU,
    AWSOrganizationOUs,
    AWSOrganizationAccount,
    AWSOrganizationAccounts
)


# Organizations limits nesting depth to 5
OU_MAX_DEPTH = 5

org = AWSOrganization.Init()

# recurse OUs to build a parent=>children tree as a dict 
def build_org_tree(parents=None, ous=None, tree=None, depth=1, maxdepth=5):
    if depth == maxdepth or len(parents) == 0:
        return {'ous': ous, 'tree': tree}
    if ous is None:
        ous = []
    if tree is None:
        tree = {}
    children = []
    print()
    print(f'OU Level: {depth}')
    print('========')
    print(f'Num Parents: {len(parents)}')
    parent_ids = str.join(" ", [parent.id for parent in parents])
    print(f'Parents: {parent_ids}')
    print()
    for parent in parents:
        # print(parent)
        parent_children = AWSOrganizationOUs.init(parent_id=parent.id).ous
        for child in parent_children:
            children.append(child)
        ous.extend(parent_children)
        tree.update({parent.id: parent_children})
    build_org_tree(parents=children, ous=ous, tree=tree, depth=depth+1)

org_data = build_org_tree([org.root])

ClientError: An error occurred (ExpiredTokenException) when calling the DescribeOrganization operation: The security token included in the request is expired

In [13]:
ou_levels = dict.fromkeys(range(1,OU_MAX_DEPTH+1))
ou_levels[1] = ous.ous
ou_levels

ou_tree_graph = {
    org.root_id: [ou.id for ou in ous.ous]
}

ou_tree_graph = {**ou_tree_graph, **{ou_id: None for ou_id in ou_tree_graph[org.root_id]}}
ou_tree_graph

{'r-iotf': ['ou-iotf-053pmb2c',
  'ou-iotf-xdkmbt53',
  'ou-iotf-mqg75o3p',
  'ou-iotf-uwhirm40',
  'ou-iotf-qltcjjwd',
  'ou-iotf-bffzgqbw',
  'ou-iotf-qo8o6fub',
  'ou-iotf-y9lhcu50',
  'ou-iotf-njh1zn1a',
  'ou-iotf-krhbch7g',
  'ou-iotf-exmifma1',
  'ou-iotf-88qyt0of',
  'ou-iotf-wjtvd5jd',
  'ou-iotf-eobj1uxe',
  'ou-iotf-gez0wmbw',
  'ou-iotf-dd4y6w9y',
  'ou-iotf-3bmi9csb'],
 'ou-iotf-053pmb2c': None,
 'ou-iotf-xdkmbt53': None,
 'ou-iotf-mqg75o3p': None,
 'ou-iotf-uwhirm40': None,
 'ou-iotf-qltcjjwd': None,
 'ou-iotf-bffzgqbw': None,
 'ou-iotf-qo8o6fub': None,
 'ou-iotf-y9lhcu50': None,
 'ou-iotf-njh1zn1a': None,
 'ou-iotf-krhbch7g': None,
 'ou-iotf-exmifma1': None,
 'ou-iotf-88qyt0of': None,
 'ou-iotf-wjtvd5jd': None,
 'ou-iotf-eobj1uxe': None,
 'ou-iotf-gez0wmbw': None,
 'ou-iotf-dd4y6w9y': None,
 'ou-iotf-3bmi9csb': None}

In [22]:
org_data

In [1]:
from aws_data_tools.models import (
    AWSOrganization,
    AWSOrganizationRoot,
    AWSOrganizationRoots,
    AWSOrganizationOU,
    AWSOrganizationOUs,
    AWSOrganizationAccount,
    AWSOrganizationAccounts
)

org = AWSOrganization()
org.build_ou_tree()

In [3]:
len(org.ous)

34

In [5]:
org.ou_tree.keys()

dict_keys(['r-iotf', 'ou-iotf-053pmb2c', 'ou-iotf-xdkmbt53', 'ou-iotf-mqg75o3p', 'ou-iotf-uwhirm40', 'ou-iotf-qltcjjwd', 'ou-iotf-bffzgqbw', 'ou-iotf-qo8o6fub', 'ou-iotf-y9lhcu50', 'ou-iotf-njh1zn1a', 'ou-iotf-krhbch7g', 'ou-iotf-exmifma1', 'ou-iotf-88qyt0of', 'ou-iotf-wjtvd5jd', 'ou-iotf-eobj1uxe', 'ou-iotf-gez0wmbw', 'ou-iotf-dd4y6w9y', 'ou-iotf-3bmi9csb', 'ou-iotf-eor9nwuw', 'ou-iotf-3l4m9zr4', 'ou-iotf-dt34apb2', 'ou-iotf-7wo3fmr8', 'ou-iotf-67kh0y04', 'ou-iotf-jc84x6x4', 'ou-iotf-5qoyxtun', 'ou-iotf-bj7ertn3', 'ou-iotf-3e39pmqs', 'ou-iotf-cdhdrv6p', 'ou-iotf-ardxws5o', 'ou-iotf-3i4b958t', 'ou-iotf-c6tyeplj', 'ou-iotf-dybvopk7', 'ou-iotf-qdg49d4m', 'ou-iotf-yg6a3r6u', 'ou-iotf-w8p7yscv'])

In [7]:
[ou.name for ou in org.ous]

['CBSi Media Brands',
 'CBS Corp',
 'Data Science Services',
 'Games',
 'CNET Tech',
 'Showtime',
 'CBS Digital Media',
 'Smithsonian',
 'Advanced Tech',
 'Hackday',
 'Simon and Schuster',
 'Revenue Operations',
 'Suspended',
 'Core Engineering',
 'Central Operations',
 'CBS Sports Digital',
 'CBS Local',
 'LastFM',
 'CBS Entertainment',
 'CBSNews',
 'CBS.com',
 'CAN',
 'CBS Television Distribution',
 'Video',
 'URS',
 'Central Services',
 'Infosec',
 'Fantasy',
 'Advanced Media',
 'CBSSports.com',
 'Sportsline',
 '247Sports',
 'MaxPreps',
 'bootstrap']

In [1]:
from aws_data_tools.models import AWSOrganization
org = AWSOrganization(init_accounts=True)

In [2]:
len(org.accounts)

171

In [3]:
org.accounts[0:10]

[AWSOrganizationAccount(arn='arn:aws:organizations::846755659841:account/o-6k4j5gipkv/104177216343', email='i-aws-atlassian-dev@cbsinteractive.com', id='104177216343', joined_method='CREATED', joined_timestamp=datetime.datetime(2020, 8, 31, 14, 55, 36, 360000, tzinfo=tzlocal()), name='i-aws-atlassian-dev', parent_id='r-iotf', status='ACTIVE', tags=None),
 AWSOrganizationAccount(arn='arn:aws:organizations::846755659841:account/o-6k4j5gipkv/644020930460', email='i-aws-adtech-prod@cbsinteractive.com', id='644020930460', joined_method='CREATED', joined_timestamp=datetime.datetime(2019, 10, 8, 11, 8, 55, 893000, tzinfo=tzlocal()), name='i-aws-adtech-prod', parent_id='r-iotf', status='ACTIVE', tags=None),
 AWSOrganizationAccount(arn='arn:aws:organizations::846755659841:account/o-6k4j5gipkv/239393157134', email='i-aws-cmgbi-prod@cbsinteractive.com', id='239393157134', joined_method='CREATED', joined_timestamp=datetime.datetime(2020, 4, 23, 10, 12, 49, 952000, tzinfo=tzlocal()), name='i-aws-cm

In [6]:
orgdict = org.as_dict()

In [7]:
orgdict['accounts']

[{'arn': 'arn:aws:organizations::846755659841:account/o-6k4j5gipkv/104177216343',
  'email': 'i-aws-atlassian-dev@cbsinteractive.com',
  'id': '104177216343',
  'joined_method': 'CREATED',
  'joined_timestamp': datetime.datetime(2020, 8, 31, 14, 55, 36, 360000, tzinfo=tzlocal()),
  'name': 'i-aws-atlassian-dev',
  'parent_id': 'r-iotf',
  'status': 'ACTIVE',
  'tags': None},
 {'arn': 'arn:aws:organizations::846755659841:account/o-6k4j5gipkv/644020930460',
  'email': 'i-aws-adtech-prod@cbsinteractive.com',
  'id': '644020930460',
  'joined_method': 'CREATED',
  'joined_timestamp': datetime.datetime(2019, 10, 8, 11, 8, 55, 893000, tzinfo=tzlocal()),
  'name': 'i-aws-adtech-prod',
  'parent_id': 'r-iotf',
  'status': 'ACTIVE',
  'tags': None},
 {'arn': 'arn:aws:organizations::846755659841:account/o-6k4j5gipkv/239393157134',
  'email': 'i-aws-cmgbi-prod@cbsinteractive.com',
  'id': '239393157134',
  'joined_method': 'CREATED',
  'joined_timestamp': datetime.datetime(2020, 4, 23, 10, 12, 49

In [8]:
org

{
    "account_tree": {
        "ou-iotf-053pmb2c": [],
        "ou-iotf-3bmi9csb": [],
        "ou-iotf-3e39pmqs": [
            {
                "arn": "arn:aws:organizations::846755659841:account/o-6k4j5gipkv/620031825250",
                "email": "i-aws-core-services@cbsinteractive.com",
                "id": "620031825250",
                "joined_method": "CREATED",
                "joined_timestamp": "2018-05-24 06:40:14.559000-05:00",
                "name": "i-aws-core-services",
                "parent_id": "ou-iotf-3e39pmqs",
                "status": "ACTIVE",
                "tags": null
            }
        ],
        "ou-iotf-3i4b958t": [
            {
                "arn": "arn:aws:organizations::846755659841:account/o-6k4j5gipkv/968282610457",
                "email": "i-aws-csports@cbsinteractive.com",
                "id": "968282610457",
                "joined_method": "INVITED",
                "joined_timestamp": "2016-08-09 08:05:43-05:00",
                "